In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from pathlib import Path
from glob import glob
from tqdm.auto import tqdm
import numpy as np
from scipy.spatial import KDTree
from matplotlib import pyplot as plt
import matplotlib
import tiledb
from tqdm.auto import tqdm
import geopandas as gpd

In [35]:
from pyspatialkit.storage.geostorage import GeoStorage
from pyspatialkit.dataobjects.geoshape import GeoShape
from pyspatialkit.dataobjects.georaster import GeoRaster
from pyspatialkit.dataobjects.geopointcloud import GeoPointCloud
from pyspatialkit.spacedescriptors.georect import GeoRect
from pyspatialkit.crs.geocrs import GeoCrs
from pyspatialkit.spacedescriptors.geobox2d import GeoBox2d

In [8]:
DATA_PATH = Path('/hpi/fs00/home/tobias.pietz/pyspatialkit/data/')
TMP_DATA_PATH = Path('/tmp/pyspatialkit/data/')
GEOSTORAGE_PATH = TMP_DATA_PATH / 'geostore/'
GEOSTORAGE_PATH.mkdir(exist_ok=True, parents=True)
DGM_PATH = DATA_PATH / 'dgm2/'
DOM_PATH = DATA_PATH / 'dom2/'
RGBI_PATH = DATA_PATH / 'rgbi100/'
CRS = GeoCrs.from_epsg(25832)
AOI = GeoShape.from_shapefile(DATA_PATH / 'aoi.shp').to_crs(CRS)
AOI_BOUNDS = AOI.bounds

In [9]:
storage = GeoStorage(GEOSTORAGE_PATH)

In [10]:
rgbi_layer = storage.add_raster_layer('rgbi', num_bands = 4, dtype=np.uint8, crs=CRS, bounds=AOI_BOUNDS,
                                      build_pyramid=True)
rgbi_layer.begin_pyramid_update_transaction()
pathlist = list(RGBI_PATH.glob('*.tif'))
for path in tqdm(pathlist):
    raster = GeoRaster.from_file(path)
    raster.data = raster.data.astype(np.uint8)
    rgbi_layer.write_data(raster)
rgbi_layer.commit_pyramid_update_transaction()

  0%|          | 0/722 [00:00<?, ?it/s]

UPDATING PYRAMIDS


In [15]:
dom_layer.backend.levels

[['/tmp/pyspatialkit/data/geostore/dom/backend/level_0', None]]

In [27]:
dom_layer.delete_permanently()

In [33]:
pc_data_scheme = {'x': np.dtype('float64'), 'y': np.dtype('float64'), 'z': np.dtype('float64')}
dom_layer = storage.add_point_cloud_layer('dom', crs=CRS, bounds=AOI_BOUNDS, data_scheme=pc_data_scheme,
                                          build_pyramid=False, point_density=1)
pathlist = list(DOM_PATH.glob('*.xyz'))
for path in tqdm(pathlist):
    pc = GeoPointCloud.from_xyz_file(path, crs=CRS)
    dom_layer.write_data(pc)

extent:[ 71271.3483359   43257.17673566 120000.        ]
size[100. 100. 100.]


  0%|          | 0/722 [00:00<?, ?it/s]

/hpi/fs00/home/tobias.pietz/pyspatialkit/pyspatialkit/pyspatialkit/storage/pointcloud/tiledbsparsebackend.py:118: UserWarning: Out of bounds write, writing only values inside of layer bounds...
  # TODO: merge bounds at every level to increase performance of batch updates


In [34]:
pc_data_scheme = {'x': np.dtype('float64'), 'y': np.dtype('float64'), 'z': np.dtype('float64')}
dgm_layer = storage.add_point_cloud_layer('dgm', crs=CRS, bounds=AOI_BOUNDS, data_scheme=pc_data_scheme,
                                          build_pyramid=False, point_density=1)
pathlist = list(DGM_PATH.glob('*.xyz'))
for path in tqdm(pathlist):
    pc = GeoPointCloud.from_xyz_file(path, crs=CRS)
    dgm_layer.write_data(pc)

extent:[ 71271.3483359   43257.17673566 120000.        ]
size[100. 100. 100.]


  0%|          | 0/722 [00:00<?, ?it/s]

In [38]:
test_bounds = (722830.9, 5747402.9, 723030.6, 5747608.7)
test_box = GeoBox2d.from_bounds(test_bounds, crs=CRS)

In [39]:
test_dom = dom_layer.get_data(test_box)

In [ ]:
test_dom.plot_o3dj()